In [1]:
# Add variable outputs, assign grid
using SpeedyWeather
using CairoMakie

In [2]:
spectral_grid = SpectralGrid(trunc = 31, nlayers=1, Grid=FullGaussianGrid)
output = NetCDFOutput(spectral_grid, path="C:/Users/Yan/OneDrive/Desktop/UNI/Honours/ml-for-da-yanny-honours-2025/one_layer_up_to_training")

@kwdef struct TempOutput <: SpeedyWeather.AbstractOutputVariable
    name::String = "temp"
    unit::String = "K"
    long_name::String = "temperature"
    dims_xyzt::NTuple{4, Bool} = (true, true, true, true)
end

@kwdef struct SurPresOutput <: SpeedyWeather.AbstractOutputVariable
    name::String = "spres"
    unit::String = "Pa"
    long_name::String = "surface pressure"
    dims_xyzt::NTuple{4, Bool} = (true, true, false, true)
end

@kwdef struct DivOutput <: SpeedyWeather.AbstractOutputVariable
    name::String = "div"
    unit::String = "s^-1"
    long_name::String = "divergence"
    dims_xyzt::NTuple{4, Bool} = (true, true, true, true)
end

@kwdef struct HumidOutput <: SpeedyWeather.AbstractOutputVariable
    name::String = "humid"
    unit::String = ""
    long_name::String = "specific humidity"
    dims_xyzt::NTuple{4, Bool} = (true, true, true, true)
end

@kwdef struct PrecipOutput <: SpeedyWeather.AbstractOutputVariable
    name::String = "precip"
    unit::String = ""
    long_name::String = "precipitation"
    dims_xyzt::NTuple{4, Bool} = (true, true, false, true)
end
#@kwdef struct PresOutput <: SpeedyWeather.AbstractOutputVariable
   # name::String = "log pres"
   # unit::String = "Pa"
   # long_name::String = "pressure"
   # dims_xyzt::NTuple{4, Bool} = (true, true, false, true)
#end


PrecipOutput

In [3]:

add!(output, TempOutput(), SurPresOutput(), DivOutput(), HumidOutput(), SpeedyWeather.PrecipitationOutput()...)
SpeedyWeather.path(::TempOutput, simulation) =
    simulation.diagnostic_variables.grid.temp_grid
SpeedyWeather.path(::SurPresOutput, simulation) =
    simulation.diagnostic_variables.grid.pres_grid
SpeedyWeather.path(::DivOutput, simulation) =
    simulation.diagnostic_variables.grid.div_grid
SpeedyWeather.path(::HumidOutput, simulation) =
    simulation.diagnostic_variables.grid.humid_grid
#SpeedyWeather.path(::PresOutput, simulation) =
    #simulation.diagnostic_variables.physics.cos_zenith

In [4]:
time_stepping = Leapfrog(spectral_grid, Δt_at_T31=Minute(30))
ocean = ConstantOceanClimatology(spectral_grid)
planet = Earth(spectral_grid, seasonal_cycle = true, equinox = DateTime(2000, 3, 20))

convection = SimplifiedBettsMiller(spectral_grid)
large_scale_condensation = ImplicitCondensation(spectral_grid)
temperature_relaxation = JablonowskiRelaxation(spectral_grid)
model = PrimitiveWetModel(spectral_grid; output, time_stepping, temperature_relaxation, planet, ocean)
orography = EarthOrography(spectral_grid)
initialize!(orography, model) 
simulation = initialize!(model,time=DateTime(2001,1,1))
# Output state vector every 6 hours = 21600 secs
simulation.model.output.output_dt = Second(21600)

21600 seconds

In [5]:
run!(simulation, period=Day(10955), output=true)#10955), output=true)

Weather is speedy: run 0001 100%|█████████| Time: 0:47:24 (911.58 years/day)m


Simulation{PrimitiveWetModel}
├ prognostic_variables::PrognosticVariables{...}
├ diagnostic_variables::DiagnosticVariables{...}
└ model::PrimitiveWetModel{...}